### Import the necessary libraries

In [3]:
import os
import pickle
import numpy as np
from tensorflow import keras
from keras import layers
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input, Softmax
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
#from keras.datasets import gtsrb
import tensorflow as tf
import numpy as np


2023-02-26 07:12:28.730622: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 07:12:29.100588: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-26 07:12:29.238248: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-26 07:12:30.129586: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [1]:
# Define the path to the dataset directory
dataset_dir = "/home/bakary/Desktop/My_GitHub/Defensive_distillation/Defensive-Distillation-as-an-Adversarial-Roustness-Algorithm/traffic-signs-data/"

# Load the training set
with open(os.path.join(dataset_dir, "train.p"), "rb") as f:
    train_data = pickle.load(f)

x_train, y_train = train_data["features"], train_data["labels"]

# Load the testing set
with open(os.path.join(dataset_dir, "test.p"), "rb") as f:
    test_data = pickle.load(f)

x_test, y_test = test_data["features"], test_data["labels"]

In [2]:
x_test.shape, y_test.shape, x_train.shape, y_train.shape

((12630, 32, 32, 3), (12630,), (34799, 32, 32, 3), (34799,))

### Define the teacher model

In [4]:
# The Teacher mode
teacher_model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(43)
])


2023-02-26 07:12:40.607969: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-26 07:12:40.608021: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bakary-Allahu-Akbar): /proc/driver/nvidia/version does not exist
2023-02-26 07:12:40.609127: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Define the defensive distillation function

In [5]:
# Define a defensive distillation function

def defensive_distillation(X, y, T=5, alpha=0.1):
    """
    Generate a distilled dataset using defensive distillation
    """
    logits = teacher_model(X) / T
    labels = tf.math.softmax(logits)
    labels_distilled = tf.math.pow(labels, 1 / alpha)
    labels_distilled = labels_distilled / tf.math.reduce_sum(labels_distilled, axis=1, keepdims=True)
    return X, labels_distilled

### Train the teacher model on the original dataset

In [6]:
# Train the teach model on the original dataset
teacher_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

teacher_model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1088/1088 [==============================] - 8s 7ms/step - loss: 1.1698 - accuracy: 0.7878
Epoch 2/10
1088/1088 [==============================] - 7s 7ms/step - loss: 0.1945 - accuracy: 0.9511
Epoch 3/10
1088/1088 [==============================] - 7s 6ms/step - loss: 0.1327 - accuracy: 0.9678
Epoch 4/10
1088/1088 [==============================] - 7s 6ms/step - loss: 0.1169 - accuracy: 0.9717
Epoch 5/10
1088/1088 [==============================] - 7s 7ms/step - loss: 0.1001 - accuracy: 0.9746
Epoch 6/10
1088/1088 [==============================] - 7s 7ms/step - loss: 0.0942 - accuracy: 0.9775
Epoch 7/10
1088/1088 [==============================] - 7s 7ms/step - loss: 0.0783 - accuracy: 0.9809
Epoch 8/10
1088/1088 [==============================] - 7s 7ms/step - loss: 0.0759 - accuracy: 0.9833
Epoch 9/10
1088/1088 [==============================] - 8s 7ms/step - loss: 0.0667 - accuracy: 0.9848
Epoch 10/10
1088/1088 [==============================] - 8s 7ms/step - loss: 0.075

### Generate a distilled dataset using defensive distillation

In [ ]:
X_train_distilled, y_train_distilled = defensive_distillation(x_train, y_train)

### Define the student model

In [ ]:
# Define the student model

student_model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(43)
])

### Train a Student model on the distilled dataset


#### Student_model = "model_distilled"

In [ ]:
student_model.compile(optimizer='adam',
                        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                        metrics=['accuracy'])

student_model.fit(X_train_distilled, y_train_distilled, epochs=10)

### Generate ensemble adversarial examples

In [ ]:
#%%
# Define the path to the dataset directory
dataset_dir = "/home/bakary/Desktop/My_GitHub/Defensive_distillation/Defensive-Distillation-as-an-Adversarial-Roustness-Algorithm/traffic-signs-data/"



# Load the testing set
with open(os.path.join(dataset_dir, "test.p"), "rb") as f:
    test_data = pickle.load(f)

x_test, y_test = test_data["features"], test_data["labels"]



# Define the epsilon values for the attacks
epsilons = [0.01, 0.02, 0.03]



# Define the FGSM attack function
def fgsm_attack(model, x, y, epsilon):
    """
    Fast Gradient Sign Method attack
    """
    x_adv = tf.Variable(x)
    with tf.GradientTape() as tape:
        tape.watch(x_adv)
        logits = model(x_adv)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y, logits)
    gradients = tape.gradient(loss, x_adv)
    perturbation = tf.sign(gradients)
    x_adv = x_adv + epsilon*perturbation
    x_adv = tf.clip_by_value(x_adv, 0, 1)
    return x_adv.numpy()



# Define the I-FGSM attack function
def ifgsm_attack(model, x, y, epsilon, alpha=0.01, num_iter=10):
    """
    Iterative Fast Gradient Sign Method attack
    """
    x_adv = tf.Variable(x)
    for i in range(num_iter):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            logits = model(x_adv)
            loss = tf.keras.losses.sparse_categorical_crossentropy(y, logits)
        gradients = tape.gradient(loss, x_adv)
        perturbation = alpha*tf.sign(gradients)
        x_adv = x_adv + perturbation
        x_adv = tf.clip_by_value(x_adv, x-epsilon, x+epsilon)
        x_adv = tf.clip_by_value(x_adv, 0, 1)
    return x_adv.numpy()



# Define the function to generate adversarial examples using an ensemble of attacks
def generate_ensemble_adversarial(model, x, y, epsilons):
    """
    Generate adversarial examples using an ensemble of FGSM and I-FGSM attacks
    """
    x_adv = x.copy()
    for epsilon in epsilons:
        x_adv_fgsm = fgsm_attack(model, x_adv, y, epsilon)
        x_adv_ifgsm = ifgsm_attack(model, x_adv, y, epsilon)
        x_adv = (x_adv + x_adv_fgsm + x_adv_ifgsm) / 3.0
    return x_adv



# Generate adversarial examples using the ensemble of attacks
x_adv = generate_ensemble_adversarial(student_model, x_test, y_test, epsilons)

# Save the adversarial examples to your directory
with open(os.path.join(dataset_dir, "adversarial.p"), "wb") as f:
    pickle.dump({"features": x_adv, "labels": y_test}, f)



### Evaluate the model on the test dataset

In [ ]:
student_model.evaluate(x_test, y_test)

### Evaluate the model with ensemble adversarial attacks.

In [ ]:
# Define the path to the adversarial examples directory
adv_examples_dir = "/home/bakary/Desktop/My_GitHub/Defensive_distillation/Defensive-Distillation-as-an-Adversarial-Roustness-Algorithm/traffic-signs-data/"

# Load the adversarial examples
with open(os.path.join(adv_examples_dir, "adversarial.p"), "rb") as f:
    adversarial_data = pickle.load(f)

x_adv, y_adv = adversarial_data["features"], adversarial_data["labels"]

# Load the student model
student_model = tf.keras.models.load_model("/home/bakary/Desktop/My_GitHub/Defensive_distillation/Defensive-Distillation-as-an-Adversarial-Roustness-Algorithm/model.h5")

# Evaluate the student model on the adversarial examples
adv_loss, adv_acc = student_model.evaluate(x_adv, y_adv)

print("Adversarial loss:", adv_loss)
print("Adversarial accuracy:", adv_acc)